In [1]:
%%capture    
import os
os.environ['FIFTYONE_DATABASE_NAME'] = 'fiftyone-teams'
import fiftyone as fo

import numpy as np
from PIL import Image
import pandas as pd
import cv2
import matplotlib.pyplot as plt ## check to use PIL (low)
from tqdm import tqdm
from torchvision import datasets, transforms, models
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os

CURRENT_DIR = os.getcwd()
PATH_TO_YOLO = f'{CURRENT_DIR}/detection/exp/train4/weights/best.pt'
YOLO_MODEL = os.path.join(CURRENT_DIR, PATH_TO_YOLO)
print(YOLO_MODEL)

/home/jovyan/BA/detection/exp/train4/weights/best.pt


In [3]:
ds = fo.load_dataset("sellpy-test")

In [4]:
#limit = 10000
# get all ids
ds_temp = ds.match(fo.ViewField("Grounding_Dino") !=  None)
print(ds_temp)
ids = ds_temp.values("id")
# check weather the sample contains a logo
contains_logo = ["logo" in values for values in ds_temp.values("Grounding_Dino.detections.label")]
# get ids of sample containing a logo 
ids_contains_logo = np.array(ids)[contains_logo]
# filter the data set to contain only samples with a logo found
ds_with_logos = ds[ids_contains_logo]

Dataset:     sellpy-test
Media type:  image
Num samples: 204676
Sample fields:
    id:             fiftyone.core.fields.ObjectIdField
    filepath:       fiftyone.core.fields.StringField
    tags:           fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    lvl0:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl1:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl2:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    lvl3:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    size:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    color:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    fabric:         fiftyon

In [7]:
brand_values = ds_with_logos.count_values("brand.label")
brands=list(brand_values.keys())
len(brands)

5177

In [8]:
import yaml

def get_class_names(yaml_file_path):
    """
    Reads the data.yaml file and extracts the class names.

    Parameters:
    yaml_file_path (str): The path to the data.yaml file.

    Returns:
    list: A list of class names.
    """
    with open(yaml_file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['names']

# Define the path to your data.yaml file
yaml_file_path = 'detection/Dataset/data.yaml'

# Extract the class names
model_classes = get_class_names(yaml_file_path)

# Print the class names
#print(f"Class names: {class_names}")


In [9]:
import re
def normalize_brands(brand_list):
    normalized = []
    for brand in brand_list:
        # Umwandlung in Kleinbuchstaben
        brand = brand.lower()
        # Entfernen aller Wörter nach dem ersten
        #clean_brand = re.sub(r'\s.*', '', brand)
        normalized.append(brand)
    return normalized

class_names = normalize_brands(model_classes)
brands_ubber = normalize_brands(brands)
matches = set(class_names).intersection(set(brands_ubber))

print(f'Total brands: {len(brands)}')
print(f'Garments-brands: {len(class_names)}')

print(f'Total images: {sum(brand_values.values())}')
# Check if there are any matches
if matches:
    print("Matches found:", len(matches))
else:
    print("No matches found.")
model_fiftyone_labels_map = {brands[brands_ubber.index(b)]:class_names.index(b) for b in matches}

idx = [brands_ubber.index(b) for b in matches]
result_dict = {brands[key]:brand_values[brands[key]] for key in idx}
print(result_dict)
print(f'Total number of images for match brands: {sum(result_dict.values())}')
print('...............................................')

Total brands: 5177
Garments-brands: 569
Total images: 37507
Matches found: 118
{"Marc O'Polo": 56, 'Missoni': 2, 'Head': 5, 'Wolford': 8, 'Champion': 171, 'Canterbury': 2, 'Topman': 8, 'Acqua Limone': 12, 'Zara': 751, 'Skins': 4, 'Akris': 1, 'Calvin Klein Underwear': 1, 'American Apparel': 5, 'Imperial': 1, 'Patagonia': 35, 'Wrangler': 19, 'Ellesse': 87, 'Obey': 7, 'Hummel': 94, 'WeSC': 8, 'Woolrich': 4, 'Napapijri': 28, 'Rocky': 1, 'Givova': 1, 'Vince Camuto': 1, 'The North Face': 168, 'RVCA': 1, 'Polo Ralph Lauren': 252, 'Barbie': 3, 'Etnies': 1, 'Lonsdale': 3, 'Timberland': 11, 'Freya': 3, 'Columbia': 18, 'And1': 2, 'Everlast': 4, 'Kiton': 1, 'La Perla': 1, 'Hugo Boss': 17, 'Calvin Klein': 149, 'Balmain': 1, 'American Eagle': 9, 'Triumph': 25, 'Adidas': 1909, 'Mexx': 33, 'Frank Dandy': 7, 'Los Angeles': 1, 'Moncler': 3, 'Under Armour': 129, 'Emilio': 5, 'Gap': 62, 'Chaps': 1, 'A.P.C': 4, 'Tommy Bahama': 2, 'Joules': 7, 'Berghaus': 3, 'Rockport': 1, 'Antony Morato': 1, 'Lacoste': 57,

In [10]:
dataset_model_classes = ds_with_logos.match(fo.ViewField("brand.label").is_in(result_dict.keys()))

In [11]:
brand_values = dataset_model_classes.count_values("brand.label")
len(brand_values)

118

In [12]:
from ultralytics import YOLO
model = YOLO(PATH_TO_YOLO') # load a custom model

In [28]:
import cv2
def show_predictions(image, results):
    result = results[0]
    for box in result.boxes:
        class_id = result.names[box.cls[0].item()]
        cords = box.xyxy[0].tolist()
        cords = [round(x) for x in cords]
        conf = round(box.conf[0].item(), 2)
        print("Object type:", class_id)
        print("Coordinates:", cords)
        print("Probability:", conf)
        print("---")
    im_array = result.plot()  # plot a BGR numpy array of predictions
    cv2.imwrite('test/predicted.png',im_array)

In [29]:
for sample in dataset_model_classes:
    img = fo.utils.image.read(sample.get_local_path("filepath"))
    results = model.predict(sample.get_local_path("filepath"),conf = 0.5, iou = 0.6, save=True)
    show_predictions(img, results)
    break


image 1/1 /home/jovyan/fiftyone/__cache__/media/minio/sellpy-images-pipeline/yQqYbiYZaD-irOWSZcGb0-c6ae-0.jpg: 640x640 (no detections), 4.2ms
Speed: 16.9ms preprocess, 4.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict26
